### 1. Neptune bulk loader

From a command line window, enter the following curl command to run the Neptune bulk loader. The key parameters that need to be modified are:
- `your-neptune-endpoint:port`
- `source`
- `format`
- `iamRoleArn`
- `region`

Modifying the other five parameters is optional. For more details, please see: https://docs.aws.amazon.com/neptune/latest/userguide/bulk-load-data.html

!curl -X POST \
    -H 'Content-Type: application/json' \
    https://your-neptune-endpoint:port/loader -d '
    {
      "source" : "s3://bucket-name/object-key-name",
      "format" : "format",
      "iamRoleArn" : "arn:aws:iam::account-id:role/role-name",
      "region" : "region",
      "failOnError" : "FALSE",
      "parallelism" : "MEDIUM",
      "updateSingleCardinalityProperties" : "FALSE",
      "queueRequest" : "TRUE",
      "dependencies" : ["load_A_id", "load_B_id"]
    }'

The above command will give the following output:
```
{
    "status" : "200 OK",
    "payload" : {
        "loadId" : "unique-ID"
    }
}
```

### 2. Get-Status request

Next, enter your `loadID` to the following curl command to get the load status. It is worth also including `details=true` and `errors=true` to the command to get a more detailed diagnosis (if there are errors).

*Note*: the `loadID` must be separated from the `details=true` and `errors=true` by a `?`

For more details, please see: https://docs.aws.amazon.com/neptune/latest/userguide/load-api-reference-status-requests.html#load-api-reference-status-request-syntax

In [ ]:
!curl -X GET https://your-neptune-endpoint:port/loader/loadId

# Include ?details=true&errors=true
!curl -X GET https://your-neptune-endpoint:port/loader/loadId?details=true&errors=true

If the loader returns `"LOAD_COMPLETED" : 1`, then queries can be performed. Otherwise, the errors need to be diagnosed. 

### 3. Gremlin querying

Gremlin queries need to be initiated with cell magic. Perform this by running `%%gremlin` at the start of each cell, before your query.

In order to query a vertex file, initiate the query with `g.V()`; and to query an edge file, initiate the query with `g.E()`.

For more detail on Gremlin, please refer to the documentation: https://tinkerpop.apache.org/docs/current/reference/

Below are some example queries:

In [ ]:
# Find all vertices with a certain label
g.V().hasLabel('person')

In [ ]:
# Find all vertices connected to a specific vertex
g.V().has('person', 'name', 'John').out()

In [ ]:
# Filter vertices based on a property value
g.V().hasLabel('person').has('age', gt(30))

In [ ]:
# Find the shortest path between two vertices
g.V().has('person', 'name', 'John').repeat(out()).until(has('person', 'name', 'Jane')).path()

In [ ]:
# Group vertices by a property value
g.V().hasLabel('person').group().by('gender')

In [ ]:
# Count the number of vertices with a certain label
g.V().hasLabel('person').count()

In [ ]:
# Sum the values of a property for a group of vertices
g.V().hasLabel('person').values('age').sum()